### Approximate string matching with FuzzyWuzzy

- FuzzyWuzzy is a python library uses Levenstein Distance to calculate the differences between sequences
- following https://github.com/seatgeek/fuzzywuzzy
- and https://towardsdatascience.com/natural-language-processing-for-fuzzy-string-matching-with-python-6632b7824c49

In [2]:
import db_conn
# import pymysql
import pandas as pd
from fuzzywuzzy import fuzz

OperationalError: could not connect to server: Connection timed out
	Is the server running on host "smith.snu.ac.kr" (147.46.174.47) and accepting
	TCP/IP connections on port 5432?


In [ ]:
# ratio compare the entire string similarity in order
fuzz.ratio('mesna', 'mesnex')
# 73 = 73% similar

In [ ]:
# partial_ratio compares partial string similarity
fuzz.partial_ratio('I love computer science', 'computer programming is what I love the most')

In [ ]:
# token_sort_ratio : ignores word order
fuzz.token_sort_ratio('I love computer science', 'computer programming is what I love the most')

In [ ]:
# token_set_ratio : ignores duplicate words
fuzz.token_set_ratio('I love computer science', 'computer programming is what I love the most')

In [ ]:
# token_set_ratio : ignores duplicate words
fuzz.token_set_ratio('I love computer science', 'computer programming')

In [ ]:
# if conn:
# conn.close()
conn = db_conn.get_connection()
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [ ]:
# get drug list from dictionary table
cursor.execute('SELECT * FROM dict_collapsed_final')
drugs = pd.DataFrame(cursor.fetchall())

In [ ]:
drugs.head(10)

## cui1 와 cui2 string을 비교하기 = 연습으로

## token_set_ratio

In [ ]:
drugs['fuzzy_perc'] = drugs.apply(lambda x: \
                                  fuzz.token_set_ratio(x['cui1_str'], x['cui2_str']),\
                                axis=1)

In [ ]:
drugs.head()

In [ ]:
# 아예 같은 약물이 아닌, 일부 스펠링이 다른 경우 어느 정도의 약물을 같은 약물로 판단할 수 있을 것인가
drugs[(drugs.fuzzy_perc>70) & (drugs.fuzzy_perc<100)][:10]

In [ ]:
drugs[(drugs.fuzzy_perc<50)][:10]

In [ ]:
# sample table -> sentence file
sents = []
with open('./new_sample_data_m.txt', 'r') as file:
    for line in file.readlines():
        if(line.find('***')<0):
            sents.append(line.strip('\n'))

In [ ]:
sents[:10]

In [ ]:
similarity_gtl_80 = []
for d in drugs.cui1_str.unique():
    for sen in sents:
        fuzz_ratio = fuzz.token_set_ratio(d, sen)
        if fuzz_ratio >= 80:
            similarity_gtl_80.append({'drug':d, 'sen':sen, 'ratio':fuzz_ratio})

In [ ]:
df_sim_gtl_80 = pd.DataFrame(similarity_gtl_80)

In [ ]:
df_sim_gtl_80.shape

In [ ]:
df_sim_gtl_80.head()

In [ ]:
pd.set_option('display.max_colwidth', -1)
df_sim_gtl_80[df_sim_gtl_80.ratio<100]

In [ ]:
drugs.loc[(drugs.cui1_str=='zinc acetate') | (drugs.cui2_str=='zinc acetate')]

In [ ]:
drugs.loc[(drugs.cui1_str=='abiraterone acetate') | (drugs.cui2_str=='abiraterone acetate')]

In [ ]:
drugs.loc[(drugs.cui1_str.str.find('sipul')>=0) | (drugs.cui2_str.str.find('sipul')>=0)]

## extract

In [ ]:
from fuzzywuzzy import process

In [ ]:
choices = ['I love computer science', 'COMPUTER SCIENCE', 'computer programming', 'programming IT']
process.extract('computer science', choices, limit=3, scorer=fuzz.token_set_ratio)

In [ ]:
similarity_gtl_80_extract = []
for d in drugs.cui1_str.unique():
    sens = process.extract(d, sents, limit=3, scorer=fuzz.token_set_ratio)
    if len(sens) >=0 :
        similarity_gtl_80_extract.append({d:sens})

In [ ]:
pairs = []
for pair in similarity_gtl_80_extract:
    d = list(pair.keys())[0]
    above_80 = list(map(lambda x: {'drug':d, 'sen':x[0], 'perc':x[1]}, filter(lambda x: x[1] > 80, pair[d])))
    if len(above_80)>0:
        pairs.extend(above_80)

In [ ]:
pd.DataFrame(pairs).head()

In [ ]:
tables_count = 6000
retrieve_strip_html = """select strip_tags(content) as c, table_title, id 
                        from research.article_tables 
                        order by id
                        limit 1000 offset %s"""

In [ ]:
import pixiedust

In [ ]:
#  %%pixie_debugger

possible_drugs = []
for c in range(70):
    print(c*1000)
    print(retrieve_strip_html % (c*1000))
    cursor.execute(retrieve_strip_html % (c*1000))
    result = list(cursor.fetchall())

    for d in drugs.cui1_str.unique():#['sipuleucel-t']:#
    #     cursor.execute(check_query % ("'%"+d[:-cut_len]+"%'", "'%"+d+"%'"))
        fuzz_r = 0
        for r in result:
            rc = r['c'].replace('\n', ' ')
            fuzz_r = fuzz.partial_token_set_ratio(d, rc)
            if fuzz_r >= 80 and fuzz_r<100:
                
                words = list(filter(lambda x:x.strip() and len(x.strip())>5, rc.split(' ')))
                best_words = process.extractBests(d, words, limit=2, scorer=fuzz.token_set_ratio)
                p_drugs = list(filter(lambda x: x[1]>=70, best_words))
                if len(p_drugs)>0:
                    print(d, r['table_title'])
                    print('\t' + str(p_drugs))
                    possible_drugs.extend(list(map(lambda x:{'drug':d, 'p_drug':x[0], 'id':r['id'], 'ratio':x[1]}, p_drugs)))

In [ ]:
df_pos_dr = pd.DataFrame(possible_drugs)
df_pos_dr.shape

In [ ]:
df_pos_dr.drop_duplicates(subset=['p_drug'], inplace=True)
df_pos_dr.shape

In [ ]:
for dr in df_pos_dr[:3].iterrows():
    print(dr[1])

In [ ]:
not_in_dict = []
for dr in df_pos_dr.iterrows():
    dr_ = '%'+dr[1]['p_drug'].lower().strip()\
            .replace('(', '').replace(',', '').replace(':', '').replace('+', '').replace(';', '')\
            .replace('.', '').replace(')', '')+'%'
    cursor.execute("select * from dict_collapsed_final where lower(cui1_str) like '%s' or lower(cui2_str) like '%s';" % (dr_, dr_))
    already_in = cursor.fetchall()
    if len(already_in) ==0 and dr_.replace('%', '') not in ['otherwise', 'distribution', 'fathers', 'other','maintenance', 'father', 'dosing' ,'maintenance']:
        print(dr_.replace('%', ''))
        not_in_dict.append({'drug':dr_.replace('%', '').strip(), 'id':dr[1]['id']})

In [ ]:
with open('possible_drugs.csv', 'w') as file:
    for d in not_in_dict:
        file.write(d[0]+'\t'+d[1])